# Import Required Libraries
Import necessary libraries including kaggle, os, and tensorflow.

In [2]:
# Import necessary libraries
%pip install kaggle tensorflow pillow numpy scipy

import kaggle
import os
import tensorflow as tf
from PIL import Image


# Increase the pixel limit
Image.MAX_IMAGE_PIXELS = None

  Using cached kaggle-1.6.17-py3-none-any.whl
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.1 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Download Dataset
Authenticate with Kaggle API and download the dataset, then extract it.

In [18]:
# Step 1: Download Dataset
try:
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files("superpotato9/dalle-recognition-dataset", path="dataset", unzip=True)
    print("Dataset downloaded and extracted successfully.")
except Exception as e:
    print("Error downloading dataset:", e)
    exit(1)

Dataset URL: https://www.kaggle.com/datasets/superpotato9/dalle-recognition-dataset


KeyboardInterrupt: 

# Preprocess Data
Use TensorFlow's ImageDataGenerator to preprocess the training and validation data.

In [4]:
# Preprocess Data

data_dir = os.path.join("dataset")

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255, horizontal_flip=True, rotation_range=15, zoom_range=0.2, validation_split=0.2
)
try:
    train_generator = datagen.flow_from_directory(
        data_dir, target_size=(128, 128), batch_size=32, class_mode='binary', subset='training'
    )
    val_generator = datagen.flow_from_directory(
        data_dir, target_size=(128, 128), batch_size=32, class_mode='binary', subset='validation'
    )
except Exception as e:
    print("Error creating data generators:", e)
    exit(1)

Found 17308 images belonging to 2 classes.
Found 4327 images belonging to 2 classes.


# Build Model
Build a sequential model using TensorFlow with Conv2D, MaxPooling2D, Flatten, Dense, and Dropout layers.

In [5]:
# Build Model
base_model = tf.keras.applications.VGG16(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train Model
Train the model using the fit method with the training and validation data.

In [6]:
# Train Model



early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

try:
    history = model.fit(
        train_generator, epochs=20, validation_data=val_generator, callbacks=[early_stopping]
    )
except Exception as e:
    print("Error during model training:", e)
    exit(1)


/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 10/541 ━━━━━━━━━━━━━━━━━━━━ 9:23 1s/step - accuracy: 0.6896 - loss: 0.6739

/opt/anaconda3/lib/python3.11/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


541/541 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8204 - loss: 0.4388

/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


541/541 ━━━━━━━━━━━━━━━━━━━━ 709s 1s/step - accuracy: 0.8204 - loss: 0.4387 - val_accuracy: 0.8539 - val_loss: 0.3464
Epoch 2/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 696s 1s/step - accuracy: 0.8476 - loss: 0.3705 - val_accuracy: 0.8449 - val_loss: 0.3526
Epoch 3/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 1921s 4s/step - accuracy: 0.8603 - loss: 0.3405 - val_accuracy: 0.8539 - val_loss: 0.3323
Epoch 4/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 702s 1s/step - accuracy: 0.8568 - loss: 0.3401 - val_accuracy: 0.8667 - val_loss: 0.3274
Epoch 5/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 707s 1s/step - accuracy: 0.8572 - loss: 0.3309 - val_accuracy: 0.8618 - val_loss: 0.3260
Epoch 6/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 712s 1s/step - accuracy: 0.8659 - loss: 0.3251 - val_accuracy: 0.8697 - val_loss: 0.3120
Epoch 7/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 713s 1s/step - accuracy: 0.8648 - loss: 0.3143 - val_accuracy: 0.8620 - val_loss: 0.3287
Epoch 8/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 712s 1s/step - accuracy: 0.8650 - loss: 0.3102 - val_accuracy: 0.86

# Save Model
Save the trained model to a file.

In [7]:
# Save Model
model.save("ai_vs_human_model.h5")
print("Model saved as ai_vs_human_model.h5")

Model saved as ai_vs_human_model.h5
